<a href="https://colab.research.google.com/github/childult-programmer/BERT/blob/main/BERT_nsmc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import os
import re
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import tqdm

from transformers import *
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", cache_dir='bert_ckpt', do_lower_case=False)

In [4]:
# 시각화
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [5]:
# random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

# 하이퍼파라미터 설정
BATCH_SIZE = 32
NUM_EPOCHS = 5
VALID_SPLIT = 0.1
MAX_LEN = 39

DATA_IN_PATH = './drive/MyDrive/NSMC/data'
DATA_OUT_PATH = './drive/MyDrive/NSMC'

## **Naver Movie Review Classification**

In [6]:
# data preprocessing
TRAIN_DATA_PATH = os.path.join(DATA_IN_PATH, "ratings_train.txt")
TEST_DATA_PATH = os.path.join(DATA_IN_PATH, "ratings_test.txt")

train_data = pd.read_csv(TRAIN_DATA_PATH, header=0, delimiter='\t', quoting=3)
train_data = train_data.dropna()
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [7]:
# special Tokens
print(tokenizer.all_special_tokens, "\n", tokenizer.all_special_ids)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'] 
 [100, 102, 0, 101, 103]


In [8]:
# Test Tokenizer
kor_encode = tokenizer.encode("안녕하세요, 반갑습니다")
eng_encode = tokenizer.encode("Hello world")

kor_decode = tokenizer.decode(kor_encode)
eng_decode = tokenizer.decode(eng_encode)

print("KR: ")
print(kor_encode)
print(kor_decode)

print("Eng: ")
print(eng_encode)
print(eng_decode)

KR: 
[101, 9521, 118741, 35506, 24982, 48549, 117, 9321, 118610, 119081, 48345, 102]
[CLS] 안녕하세요, 반갑습니다 [SEP]
Eng: 
[101, 31178, 11356, 102]
[CLS] Hello world [SEP]


In [9]:
# BERT Tokenizer
def bert_tokenizer(sent, MAX_LEN):
    encoded_dict = tokenizer.encode_plus(
        text = sent,
        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
        max_length = MAX_LEN,  # Pad & truncate all sentences
        padding='max_length',
        return_attention_mask = True, # Construct attention mask
        truncation=True
    )
    
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask'] # And its attention mask (simply differentiates padding from non-padding)
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences

    return input_id, attention_mask, token_type_id

In [10]:
input_ids = []
attention_masks = []
token_type_ids = []
train_data_labels = []

def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", " ", sent)
    return sent_clean

for train_sent, train_label in zip(train_data['document'], train_data['label']):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(clean_text(train_sent), MAX_LEN)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        train_data_labels.append(train_label)

    except Exception as e:
        print(e)
        print(train_sent)
        pass

train_movie_input_ids = np.array(input_ids, dtype=int)
train_movie_attention_masks = np.array(attention_masks, dtype=int)
train_movie_type_ids = np.array(token_type_ids, dtype=int)
train_movie_inputs = (train_movie_input_ids, train_movie_attention_masks, train_movie_type_ids)

train_data_labels = np.asarray(train_data_labels, dtype=np.int32) # Label tokenizing list

print("# sents: {}, # labels: {}".format(len(train_movie_input_ids), len(train_data_labels)))

# sents: 149995, # labels: 149995


In [11]:
input_id = train_movie_input_ids[1]
attention_mask = train_movie_attention_masks[1]
token_type_id = train_movie_type_ids[1]

print(input_id)
print(attention_mask)
print(token_type_id)
print(tokenizer.decode(input_id))

[   101    100   9928  58823  30005  11664   9757 118823  30858  18227
 119219   9580  41605  25486  12310  20626  23466   8843 118986  12508
   9523  17196  16439    102      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
[CLS] [UNK] 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [12]:
class TFBertClassifier(Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBertClassifier, self).__init__()

        self.bert = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.dropout = Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = Dense(num_class, kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range),
                                name="classifier")
        

    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        # outputs value: # sequence_output, pooled_output, (hidden_states), (attentions)
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

cls_model = TFBertClassifier(model_name='bert-base-multilingual-cased',
                             dir_path='bert_ckpt', num_class=2)

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [13]:
optimizer = tf.keras.optimizers.Adam(3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

cls_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [14]:
model_name = "bert_naver_movie"

# overfitting 방지
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=2)

checkpoint_path = os.path.join(DATA_OUT_PATH, model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Floder create complete \n".format(checkpoint_dir))

cp_callback = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1,
                              save_best_only=True, save_weights_only=True)

./drive/MyDrive/NSMC/bert_naver_movie -- Folder already exists 



In [15]:
# model training
history = cls_model.fit(train_movie_inputs, train_data_labels, epochs=NUM_EPOCHS,
                    batch_size=BATCH_SIZE, validation_split=VALID_SPLIT,
                    callbacks=[earlystop_callback, cp_callback])

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fbb85f95e58> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fbb85f95e58> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


4219/4219 [==============================] - ETA: 0s - loss: 0.4832 - accuracy: 0.7546

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


4219/4219 [==============================] - 867s 197ms/step - loss: 0.4832 - accuracy: 0.7546 - val_loss: 0.4066 - val_accuracy: 0.8033

Epoch 00001: val_accuracy improved from -inf to 0.80327, saving model to ./drive/MyDrive/NSMC/bert_naver_movie/weights.h5
Epoch 2/5
4219/4219 [==============================] - 825s 196ms/step - loss: 0.3376 - accuracy: 0.8489 - val_loss: 0.3466 - val_accuracy: 0.8545

Epoch 00002: val_accuracy improved from 0.80327 to 0.85447, saving model to ./drive/MyDrive/NSMC/bert_naver_movie/weights.h5
Epoch 3/5
4219/4219 [==============================] - 825s 196ms/step - loss: 0.2815 - accuracy: 0.8795 - val_loss: 0.3280 - val_accuracy: 0.8525

Epoch 00003: val_accuracy did not improve from 0.85447
Epoch 4/5
4219/4219 [==============================] - 825s 196ms/step - loss: 0.2366 - accuracy: 0.9017 - val_loss: 0.3758 - val_accuracy: 0.8509

Epoch 00004: val_accuracy did not improve from 0.85447


In [15]:
plot_graphs(history, 'loss')

In [16]:
# Korean Movie Review Test 데이터

In [17]:
test_data = pd.read_csv(TEST_DATA_PATH, header=0, delimiter='\t', quoting=3)
test_data = test_data.dropna()
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [22]:
nput_ids = []
attention_masks = []
token_type_ids = []
test_data_labels = []

for test_sent, test_label in zip(test_data["document"], test_data["label"]):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(test_sent, MAX_LEN)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        test_data_labels.append(test_label)
    except Exception as e:
        print(e)
        print(test_sent)
        pass

test_movie_input_ids = np.array(input_ids, dtype=int)
test_movie_attention_masks = np.array(attention_masks, dtype=int)
test_movie_type_ids = np.array(token_type_ids, dtype=int)
test_movie_inputs = (test_movie_input_ids, test_movie_attention_masks, test_movie_type_ids)

test_data_labels = np.asarray(test_data_labels, dtype=np.int32) # 레이블 토크나이징 리스트

print("num sents, labels {}, {}".format(len(test_movie_input_ids), len(test_data_labels)))

num sents, labels 49997, 49997


In [23]:
results = cls_model.evaluate(test_movie_inputs, test_data_labels, batch_size=512)
print('test loss, test acc: ', results)

98/98 [==============================] - 64s 654ms/step - loss: 0.3776 - accuracy: 0.8461
test loss, test acc:  [0.37764987349510193, 0.8460507392883301]
